In [39]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd, mysql.connector
import xgboost, textblob, string
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

from keras.layers import Embedding, Flatten, Dense
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [6]:
df1 = pd.read_excel('\\Users\\Daniel\\Python\\Data\\Flix_products\\SamsungReport.xlsx','Features', keep_default_na=False)

In [7]:
df1[['title', 'html']] = df1[['title', 'html']].astype(str)
df1.head()

,product_id,title,html
0,628714,xbox-supertemplate,xbox-supertemplate1111 kieskrreurig
1,653143,contrast enhancer,Applies one million criteria for contrast yet ...
2,653143,colour optimiser,Preserves the original level of brightness whi...
3,653143,detail enhancer,Selectively sharpens edges so that you see thi...
4,653143,3way burn-in protection,3 Types of Anti-Burn Program\nScreen burn-in i...


In [8]:
df1.apply(lambda x: x.count(), axis=0)

product_id    49215
title         49215
html          49215
dtype: int64

In [9]:
# concatenate title and html for the same product_id into one cell
df2 = df1.groupby('product_id')['title','html'].agg(lambda x: ' '.join(x)).reset_index()
df2['texts'] = df2['title'] + " " + df2['html']
df3 = df2.groupby('product_id')['texts'].agg(lambda x: ' '.join(x)).reset_index()

In [10]:
df3.shape

(6036, 2)

In [11]:
df3.head(2)

,product_id,texts
0,628714,xbox-supertemplate xbox-supertemplate1111 kies...
1,653143,contrast enhancer colour optimiser detail enha...


In [14]:
# merge some product details
products = pd.read_csv('\\Users\\Daniel\\Python\\Data\\products.csv', sep='\t')
df3 = pd.merge(df3, products[['product_id', 'flix_parent_category', 'flix_subCategory1']],
                 on='product_id', how='left')

In [16]:
df3.astype(str).groupby('flix_parent_category')['product_id'].count()

flix_parent_category
Cameras                      280
Computing/Gaming            1461
Health & Beauty                2
Home Appliances              757
Home Entertainment          1110
Office Needs                   4
Other                          7
Phone/Mobiles               1519
Small Gadgets/Appliances      21
nan                          875
Name: product_id, dtype: int64

In [17]:
# drop products with unassigned categories
df3.dropna(subset=['flix_parent_category'], inplace=True)

In [34]:
# separate description and product categories
texts = df3['texts']
texts.shape, labels.shape

((5161,), (5161,))

In [18]:
maxlen = 100   # cuts off after 100 words
training_samples = 4000    # trains on 200 samples
#validation_samples = 10000   #  validates on 10,000 samples
max_words = 1000    # considers only the top 10,000 words in the dataset

In [23]:
# converts each description to a list of integers considering only the max_words
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)  

In [24]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 15707 unique tokens.


In [46]:
# encode the target labels
encoder = preprocessing.LabelEncoder()
labels = encoder.fit_transform(df3['flix_parent_category'])
set(labels)

{0, 1, 2, 3, 4, 5, 6, 7, 8}

In [53]:
data = pad_sequences(sequences, maxlen=maxlen)   # each  tokenized review is cut to maxlen (100 words or padded 0 if less)

In [54]:
labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (5161, 100)
Shape of label tensor: (5161,)


In [51]:
indices = np.arange(data.shape[0])  
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

array([190,  16, 319,   1,   7,   2, 601,   9,  58,   9,  58,  58,  32,
        58, 198, 211, 182, 190,   3,  34,  30,  67,  19, 260, 267, 731,
        38, 692, 187,   2, 206,  38,   1, 271,  48,  80,  94, 702, 453,
        93,  70,   9, 279,  32,  96,   2,  63, 945,   3, 279,  32,  96,
       172, 382,  51, 791,   2, 274,  18,   5, 669,  58, 287, 444, 421,
        17, 601, 345,  16, 386, 126,  66, 344, 158,   1,  63, 570,  51,
       218,   2,   7, 338, 401,   2, 844, 507, 158, 298, 425,  67, 519,
       693,   6,  47, 819,  67, 750, 693,   6,  47])

In [56]:
x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: ]
y_val = labels[training_samples: ]

In [60]:
# Training the same model without pretrained word embeddings
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

embedding_dim = 100
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(9, activation='softmax'))
model.summary()

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val, y_val))
model.save_weights('from scratch_model1')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 100)          100000    
_________________________________________________________________
flatten_2 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                320032    
_________________________________________________________________
dense_4 (Dense)              (None, 9)                 297       
Total params: 420,329
Trainable params: 420,329
Non-trainable params: 0
_________________________________________________________________


ValueError: Error when checking target: expected dense_4 to have shape (9,) but got array with shape (1,)